In [1]:
from captum.attr import IntegratedGradients
import numpy as np
from constants import TASK_MODEL_PATH, TASK_PATH
from scoring import load_model, load_dataset
from pathlib import Path
CUDA_DEVICE = 3
from fairseq.models.bart import BARTModel
from fairseq.data import Dictionary

/home/jgut/miniconda3/envs/SoS3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/jgut/miniconda3/envs/SoS3/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
model_path =  Path("/data/SoS_models/")/"bbbp"/"smiles_atom"/"5e-05_0.3_based_norm"/"5e-05_0.3_based_norm"/"checkpoint_last.pt"
data_path = TASK_PATH/"bbbp"/"smiles_atom"

model = BARTModel.from_pretrained(
    str(model_path.parent),
    data_name_or_path=str(data_path),
    checkpoint_file=str(model_path.name),
    layernorm_embedding=True,
)
model.to("cpu")
model.eval()
model.zero_grad()

In [3]:
data_path = TASK_PATH / "bbbp"/"smiles_atom" / "input0" / "test"
source_dictionary = Dictionary.load(str(data_path.parent / "dict.txt"))
dataset = load_dataset(data_path)
dataset[0]

tensor([ 5,  5,  9,  5,  6, 11,  9,  7,  3,  6,  5,  5,  4,  8,  4,  4,  4,  4,
         4,  8,  7, 12,  3,  6,  5,  7,  5,  6, 11,  9,  7, 12,  8,  5,  5,  5,
         3,  8,  5,  6, 11,  9,  7,  9,  2])

In [4]:
tokens = dataset[0].unsqueeze(-1)
prev_output_tokens = tokens.clone()

prev_output_tokens[:, 0] = tokens.gather(
    0,
    (tokens.ne(source_dictionary.pad()).sum(0)-1).unsqueeze(-1)
).squeeze()

prev_output_tokens[:, 1:] = tokens[:, :-1]
# same as in predict
attention = model.model(dataset[0].unsqueeze(0), None, prev_output_tokens)[1]["attn"][0][0][0]
attention

tensor([0.0246, 0.0210, 0.0196, 0.0265, 0.0154, 0.0214, 0.0214, 0.0208, 0.0295,
        0.0160, 0.0239, 0.0230, 0.0213, 0.0262, 0.0206, 0.0216, 0.0202, 0.0211,
        0.0204, 0.0263, 0.0196, 0.0243, 0.0281, 0.0209, 0.0245, 0.0232, 0.0268,
        0.0187, 0.0222, 0.0197, 0.0211, 0.0205, 0.0285, 0.0239, 0.0227, 0.0251,
        0.0310, 0.0252, 0.0208, 0.0173, 0.0171, 0.0187, 0.0194, 0.0201, 0.0198],
       grad_fn=<SelectBackward0>)